> Tic-Tac-Toe game using MinMax Algorithm

In [2]:
import json

In [3]:
def isTerminal(state):
    combinations = [(0, 1, 2), (3, 4, 5), (6, 7, 8),  # Rows
                    (0, 3, 6), (1, 4, 7), (2, 5, 8),  # Columns
                    (0, 4, 8), (2, 4, 6)]             # Diagonals

    for combo in combinations:
        if state[combo[0]] == state[combo[1]] == state[combo[2]] == 'X':
            return True, 1                            # It's a win
        elif state[combo[0]] == state[combo[1]] == state[combo[2]] == 'O':
            return True, -1                           # It's a loss
    if state.count('_') == 0:
        return True, 0                                # Draw
    else:
        return False, 0                               # not a terminal state

In [4]:
def determineTurn(state):
    """Return X if it's max's turn otherwise O."""
    num = state.count('_')
    if num % 2 != 0:
        return 'X'
    else:
        return 'O'

In [5]:
def toString(statelst):
    """Convert the statelst(list) into a string"""
    s = ''  # initialize
    for i in statelst:
        s += i
    return s

> Creating Game tree

In [6]:
childParent = {}         # child as key, parent as value
terminals = []           # all the terminals
tree = {}                # game tree(nodes as keys, children as value)
stateScores = {}         # Storing scores of every state and turn of player
def generateTree(state):
    strState = toString(state)
    tree[strState] = []
    turn = determineTurn(state)

    if not isTerminal(state)[0]:
        for i in range(9):
            cState = state.copy()                            # copy for temparary use
            if state[i] == '_':
                cState[i] = turn
                cStateStr = toString(cState)                 # convert state into string
                tree[strState].append(cStateStr)             # add the state to tree

                if cStateStr not in childParent.keys():
                    childParent[cStateStr] = []              # add to childParent

                if strState not in childParent[cStateStr]:
                    childParent[cStateStr].append(strState)  # update childParent
                score = isTerminal(cStateStr)[1]
                stateScores[cStateStr] = [score, turn]       # add score and turn
                generateTree(cState)                         # recursive call (DFS)
    else:
        if strState not in terminals:                        # adding state to terminals
            terminals.append(strState)

generateTree(['_']*9)                                        # initial call
childParent['_________'] = [None]                            # parent of initial state
stateScores['_________'] = [0, 'X']                          # score and turn for initial state

In [7]:
def _helperUpdateValues(parents):
    while parents[0] is not None:
        for j in parents:
            scores = []
            children = tree[j]
            for k in children:
                scores.append(stateScores[k][0])
            if stateScores[j][1] == 'X':
                stateScores[j][0] = min(scores)
            else:
                stateScores[j][0] = max(scores)
            parents = childParent[j]        # find its parent(update parent)
            _helperUpdateValues(parents)    # recursive call

def updateValues():
    """Update values of the states"""
    for i in terminals:
        parents = childParent[i]            # Accessing all the parents
        _helperUpdateValues(parents)

updateValues()

In [8]:
# Storing the tree in a json file
jsonContent = {
    "tree": tree,
    "scores": stateScores
}
jsonData = json.dumps(jsonContent, indent=2)
with open('minmax.json', 'w') as json_file:
  json_file.write(jsonData)

In [9]:
def findMove(state):
    """Return the index of the best move of the computer."""
    state = toString(state)
    children = tree[state]                   # find children
    scoreChild = {}                          # scores as keys, children states as values
    for i in children:
        scoreChild[stateScores[i][0]] = i

    minScore = min(scoreChild.keys())        # minimum score among all children
    desired_state = scoreChild[minScore]     # state with min value
    for i in range(9):
        if state[i] == '_' and desired_state[i] != '_':
            return i

In [10]:
def _PrintGame(state):
    """Display the state given on the board"""
    for i in range(0, 9, 3):
      print(f"| {state[i]} | {state[i+1]} | {state[i+2]} |")
    print(" ")

In [11]:
def _declareResult(state):
    """prints the final result"""
    terminalvalue = isTerminal(state)
    if terminalvalue[1] == -1:                       # machine won
        print("You Lost")
    elif terminalvalue[1] == 1:                      # player won
        print("Congratulations! You won")
    elif terminalvalue[0] and terminalvalue[1] == 0: # ended in draw
        print("Its a draw")


In [12]:
def Play():
    """Function to play the above trained game using minmax algo"""
    state = ['_']*9                                  # initial state
    _PrintGame(state)                                # prints initial board game
    userTurn = True                                  # true if it's players turn
    indices = []                                     # played moves indices
    while not(isTerminal(state)[0]):
        if userTurn:
            try:
              index = int(input("Enter the move position(1,2,3,4,5,6,7,8,9): "))-1
            except:
              index = int(input("Please enter a valid integer: "))-1
            # asks for valid input till it is entered
            while index in indices:
              index = int(input("Please enter a valid(empty) position: "))-1
            indices.append(index)
            state[index] = 'X'
            userTurn = False
        else:
            index = findMove(state)
            indices.append(index)
            state[index] = 'O'
            userTurn = True
            _PrintGame(state)
        _declareResult(state)

Play()

| _ | _ | _ |
| _ | _ | _ |
| _ | _ | _ |
 
Enter the move position(1,2,3,4,5,6,7,8,9): 2
| _ | X | _ |
| _ | _ | _ |
| _ | O | _ |
 
Enter the move position(1,2,3,4,5,6,7,8,9): 6
| _ | X | O |
| _ | _ | X |
| _ | O | _ |
 
Enter the move position(1,2,3,4,5,6,7,8,9): 1
| X | X | O |
| _ | _ | X |
| O | O | _ |
 
Enter the move position(1,2,3,4,5,6,7,8,9): 5
| X | X | O |
| _ | X | X |
| O | O | O |
 
You Lost
